<h3 style="text-align: center; font-family: Arial, sans-serif; color: #4CAF50;">TSS - Temporal Similarity Search : POC SMALL DATASET WITH API</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
</ul>


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Charger les données
df = pd.read_csv("../Datasources/MetroPT3_imputed_final.csv", delimiter=",", decimal=".", index_col=0)
df.reset_index(drop=True, inplace=True)

# Convertir la colonne timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Sélection des colonnes continues et catégoriques
continuous_features = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]
categorical_features = ["COMP", "DV_eletric", "Towers", "LPS", "Pressure_switch", "Oil_level", "Caudal_impulses"]
columns_to_keep = ["timestamp", "panne"] + continuous_features + categorical_features

# Filtrer les colonnes utiles
df = df[columns_to_keep]


In [ ]:
display(df.head(3))

In [2]:
###################################################################################################
################ Train : Panne1   (1J ET DEMI)                                              #######
################ Test  : Panne4  (15mn + 15mn + 15mn)     #########################################
###################################################################################################

# Classe 0 : Pas de panne détectée
# Classe 1 : En pleine panne
# Classe 2 : Panne prévue dans moins de 30 minutes

pannes = [
    {'start': '2020-04-18 00:00', 'end': '2020-04-18 23:59'},
    {'start': '2020-05-29 23:30', 'end': '2020-05-30 06:00'},
    {'start': '2020-06-05 10:00', 'end': '2020-06-07 14:30'},
    {'start': '2020-07-15 14:30', 'end': '2020-07-15 19:00'},
         ]


# Définir les périodes d'entraînement et de test
train_periods = [{'start': '2020-04-17 21:30:00', 'end': '2020-04-19 00:15:00'}]
test_periods  = [{'start': '2020-07-15 12:00:00', 'end': '2020-07-15 20:00:00'}]

# Séparer les données pour l'entraînement
train_df = pd.concat([
    df[(df['timestamp'] >= pd.Timestamp(period['start'])) & 
       (df['timestamp'] <= pd.Timestamp(period['end']))]
    for period in train_periods
])

# Séparer les données pour le test
test_df = pd.concat([
    df[(df['timestamp'] >= pd.Timestamp(period['start'])) & 
       (df['timestamp'] <= pd.Timestamp(period['end']))]
    for period in test_periods
])

print(f"Entrainement : {len(train_df)} lignes")
print(f"Test : {len(test_df)} lignes")

Entrainement : 9631 lignes
Test : 2881 lignes


In [3]:
# Distribution des modalités de la variable 'panne' dans l'ensemble d'entraînement
print("Modalités de 'panne' dans l'ensemble d'entraînement :")
print(train_df["panne"].value_counts())

# Distribution des modalités de la variable 'panne' dans l'ensemble de test
print("\nModalités de 'panne' dans l'ensemble de test :")
print(test_df["panne"].value_counts())


Modalités de 'panne' dans l'ensemble d'entraînement :
1    8635
0     816
2     180
Name: panne, dtype: int64

Modalités de 'panne' dans l'ensemble de test :
1    1621
0    1080
2     180
Name: panne, dtype: int64


In [4]:
# Paramètres des fenêtres
window_size = 100  # Taille de la fenêtre
step_size = 1     # Pas de glissement
numeric_columns = continuous_features

# Générer les fenêtres pour l'entraînement
train_windows = []
for i in tqdm(range(0, len(train_df) - window_size + 1, step_size)):
    window_data = train_df.iloc[i:i + window_size][numeric_columns].values.flatten()
    #panne_value = train_df.iloc[i + window_size - 1]["panne"]          # Dernière valeur de panne dans la fenêtre
    panne_value  = train_df.iloc[i:i + window_size]["panne"].mode()[0]  # Classe majoritaire
    train_windows.append({
        "index": i,
        "timestamp": train_df.iloc[i]["timestamp"],
        "sensor_data": window_data.tolist(),
        "panne": panne_value
    })

# Générer les fenêtres pour le test
test_windows = []
for i in tqdm(range(0, len(test_df) - window_size + 1, step_size)):
    window_data = test_df.iloc[i:i + window_size][numeric_columns].values.flatten()
    #panne_value  = test_df.iloc[i + window_size - 1]["panne"]           # Dernière valeur de panne dans la fenêtre
    panne_value   = test_df.iloc[i:i + window_size]["panne"].mode()[0]  # Classe majoritaire
    test_windows.append({
        "index": i,
        "timestamp": test_df.iloc[i]["timestamp"],
        "sensor_data": window_data.tolist(),
        "panne": panne_value
    })

print(f"Fenêtres d'entraînement : {len(train_windows)}")
print(f"Fenêtres de test : {len(test_windows)}")


100%|█████████████████████████████████████████████████████████████████████████████| 2782/2782 [00:03<00:00, 758.24it/s]

Fenêtres d'entraînement : 9532
Fenêtres de test : 2782


In [6]:
import kdbai_client as kdbai

# Connexion à KDB.AI
session = kdbai.Session(endpoint="https://cloud.kdb.ai/instance/atr0v2owym", api_key="a5b897a7b4-Av/lT//S3W++g8K11vVKfJo5rIj5gK492sbpts/1f+cr/5m5n5JluPcqcQR2gGiiQQ7SrCFAc7ma2RhT")
db = session.database('default')



In [7]:
##################################
###### SUPPRIMER LA TABLE ######
##################################
# Accéder aux tables existantes
train_table = db.table("train_sensors")

try:
    train_table.drop()
    print("Table 'train_sensors' supprimée avec succès.")
except Exception as e:
    print(f"Erreur lors de la suppression de 'train_sensors' : {e}")

Table 'train_sensors' supprimée avec succès.


In [8]:

# Schéma de la table
schema = [
    {"name": "index", "type": "int64"},
    {"name": "timestamp", "type": "datetime64[ns]"},
    {"name": "sensor_data", "type": "float64s"},
    {"name": "panne", "type": "int64"}
]
indexes = [
    {"name": "flat_index", "type": "flat", "column": "sensor_data", "params": {"metric": "L2","dims": 500}}
]

# Création de la table
train_table = db.create_table("train_sensors", schema=schema, indexes=indexes)

# Insertion des données d'entraînement
batch_size = 100
for i in tqdm(range(0, len(train_windows), batch_size)):
    batch = train_windows[i:i + batch_size]
    train_table.insert(batch)

100%|██████████████████████████████████████████████████████████████████████████████████| 96/96 [00:45<00:00,  2.10it/s]


In [9]:
################################################################
#### Prédiction pour les 100 premières fenêtres             ####
################################################################

# Limiter les tests aux 100 premières fenêtres
test_subset = test_windows[:100]
results = []

for i, test_window in tqdm(enumerate(test_subset), total=len(test_subset)):
    query_vector = test_window["sensor_data"]

    try:
        # Vérifier le vecteur de requête
        if not query_vector or not isinstance(query_vector, list):
            raise ValueError(f"Vecteur de test invalide pour la fenêtre {i}: {query_vector}")

        # Recherche vectorielle
        results_vector = train_table.search(
            vectors={"flat_index": [query_vector]},
            n=5  # Trouver les 5 motifs les plus similaires
        )

        # Afficher la réponse brute pour diagnostic
        print(f"Réponse brute pour la fenêtre {i}: {results_vector}")

        # Vérifier si des résultats sont retournés
        if results_vector and isinstance(results_vector[0], list):
            similar_pannes = [result.get("panne", None) for result in results_vector[0]]
            similar_pannes = [p for p in similar_pannes if p is not None]

            if similar_pannes:
                predicted_panne = max(set(similar_pannes), key=similar_pannes.count)
            else:
                predicted_panne = -1  # Valeur par défaut si aucune panne valide
        else:
            raise ValueError(f"Aucun résultat valide pour la fenêtre {i}: {results_vector}")

        # Stocker les résultats
        results.append({
            "fenêtre": i,
            "prédiction": predicted_panne,
            "réelle": test_window["panne"]
        })

    except Exception as e:
        print(f"Erreur lors de la prédiction pour la fenêtre {i}: {e}")
        results.append({
            "fenêtre": i,
            "prédiction": -1,  # Valeur par défaut en cas d'erreur
            "réelle": test_window["panne"]
        })

# Créer un DataFrame avec les résultats
results_df = pd.DataFrame(results)

# Calculer la précision globale
accuracy = (results_df["prédiction"] == results_df["réelle"]).mean()

# Compter les fenêtres par classe prédite
class_counts = results_df["prédiction"].value_counts()

# Afficher les résultats
print(f"Précision globale pour les 100 premières fenêtres : {accuracy:.2%}")
print(f"Nombre total de fenêtres testées : {len(results_df)}")
print("\nPrédictions par classe :")
print(class_counts)

# Afficher les premiers résultats pour analyse
print(results_df.head(10))


  2%|█▋                                                                                | 2/100 [00:00<00:19,  5.04it/s]

Réponse brute pour la fenêtre 0: [   __nn_distance  index           timestamp  \
0    5033.519043   4092 2020-04-18 08:52:00   
1    5064.088867   4086 2020-04-18 08:51:00   
2    5083.428711   4098 2020-04-18 08:53:00   
3    5101.472656   1438 2020-04-18 01:29:40   
4    5106.284668   1177 2020-04-18 00:46:10   

                                         sensor_data  panne  
0  [7.154, 1.5579999999999998, 74.62500000000001,...      1  
1  [8.976, 2.004, 73.95, 5.7925, 8.836, 8.18, 1.7...      1  
2  [6.316000000000001, 1.25, 74.62500000000001, 5...      1  
3  [7.859999999999999, 1.6920000000000002, 74.25,...      1  
4  [8.542, 1.9400000000000013, 73.475, 5.6525, 8....      1  ]
Erreur lors de la prédiction pour la fenêtre 0: Aucun résultat valide pour la fenêtre 0: [   __nn_distance  index           timestamp  \
0    5033.519043   4092 2020-04-18 08:52:00   
1    5064.088867   4086 2020-04-18 08:51:00   
2    5083.428711   4098 2020-04-18 08:53:00   
3    5101.472656   1438 2020-04-

  4%|███▎                                                                              | 4/100 [00:00<00:17,  5.54it/s]

Réponse brute pour la fenêtre 2: [   __nn_distance  index           timestamp  \
0    5041.966797   4094 2020-04-18 08:52:20   
1    5084.498047   4088 2020-04-18 08:51:20   
2    5092.463379   4100 2020-04-18 08:53:20   
3    5110.592773   4093 2020-04-18 08:52:10   
4    5123.676758   1440 2020-04-18 01:30:00   

                                         sensor_data  panne  
0  [8.828, 1.9559999999999995, 74.925, 5.70000000...      1  
1  [8.934, 1.9879999999999995, 74.2, 5.7600000000...      1  
2  [8.948, 1.99, 74.62500000000001, 5.77250000000...      1  
3  [7.818, 1.661999999999999, 74.62500000000001, ...      1  
4  [8.906, 2.004, 74.64999999999999, 5.7525000000...      1  ]
Erreur lors de la prédiction pour la fenêtre 2: Aucun résultat valide pour la fenêtre 2: [   __nn_distance  index           timestamp  \
0    5041.966797   4094 2020-04-18 08:52:20   
1    5084.498047   4088 2020-04-18 08:51:20   
2    5092.463379   4100 2020-04-18 08:53:20   
3    5110.592773   4093 2020-04-

  6%|████▉                                                                             | 6/100 [00:01<00:16,  5.77it/s]

...

Réponse brute pour la fenêtre 94: [   __nn_distance  index           timestamp  \
0    4270.296387   4915 2020-04-18 11:09:10   
1    4276.761719   4914 2020-04-18 11:09:00   
2    4277.276367   4909 2020-04-18 11:08:10   
3    4279.369629   4641 2020-04-18 10:23:30   
4    4279.651855   4913 2020-04-18 11:08:50   

                                         sensor_data  panne  
0  [9.018, 2.0279999999999987, 76.29999999999998,...      1  
1  [9.016, 2.026, 76.375, 5.674999999999999, 8.88...      1  
2  [9.07, 2.042, 75.47500000000001, 5.84500000000...      1  
3  [8.92, 1.9960000000000004, 75.6, 5.78750000000...      1  
4  [9.012, 2.022, 76.42500000000001, 5.7749999999...      1  ]
Erreur lors de la prédiction pour la fenêtre 94: Aucun résultat valide pour la fenêtre 94: [   __nn_distance  index           timestamp  \
0    4270.296387   4915 2020-04-18 11:09:10   
1    4276.761719   4914 2020-04-18 11:09:00   
2    4277.276367   4909 2020-04-18 11:08:10   
3    4279.369629   4641 2020-

 98%|███████████████████████████████████████████████████████████████████████████████▍ | 98/100 [00:17<00:00,  5.72it/s]

Réponse brute pour la fenêtre 96: [   __nn_distance  index           timestamp  \
0    4292.637695   4917 2020-04-18 11:09:30   
1    4296.742188   4911 2020-04-18 11:08:30   
2    4297.574707   4916 2020-04-18 11:09:20   
3    4297.848633   4915 2020-04-18 11:09:10   
4    4306.133789   4914 2020-04-18 11:09:00   

                                         sensor_data  panne  
0  [8.988, 2.014, 76.025, 5.815, 8.86, 9.05, 2.03...      1  
1  [8.74, 1.942, 75.925, 5.717499999999999, 8.882...      1  
2  [8.082, 1.751999999999999, 76.125, 5.600000000...      1  
3  [9.018, 2.0279999999999987, 76.29999999999998,...      1  
4  [9.016, 2.026, 76.375, 5.674999999999999, 8.88...      1  ]
Erreur lors de la prédiction pour la fenêtre 96: Aucun résultat valide pour la fenêtre 96: [   __nn_distance  index           timestamp  \
0    4292.637695   4917 2020-04-18 11:09:30   
1    4296.742188   4911 2020-04-18 11:08:30   
2    4297.574707   4916 2020-04-18 11:09:20   
3    4297.848633   4915 2020-

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.67it/s]

Réponse brute pour la fenêtre 98: [   __nn_distance  index           timestamp  \
0    4332.252441   4917 2020-04-18 11:09:30   
1    4333.320312   4913 2020-04-18 11:08:50   
2    4339.135254   4919 2020-04-18 11:09:50   
3    4339.236816   4916 2020-04-18 11:09:20   
4    4343.713379   4918 2020-04-18 11:09:40   

                                         sensor_data  panne  
0  [8.988, 2.014, 76.025, 5.815, 8.86, 9.05, 2.03...      1  
1  [9.012, 2.022, 76.42500000000001, 5.7749999999...      1  
2  [9.056, 2.0380000000000003, 75.825, 5.82750000...      1  
3  [8.082, 1.751999999999999, 76.125, 5.600000000...      1  
4  [9.05, 2.033999999999999, 75.975, 5.8174999999...      1  ]
Erreur lors de la prédiction pour la fenêtre 98: Aucun résultat valide pour la fenêtre 98: [   __nn_distance  index           timestamp  \
0    4332.252441   4917 2020-04-18 11:09:30   
1    4333.320312   4913 2020-04-18 11:08:50   
2    4339.135254   4919 2020-04-18 11:09:50   
3    4339.236816   4916 2020-

In [ ]:
################################################################
#### Prédiction pour les 100 premières fenêtres             ####
################################################################

# Limiter les tests aux 100 premières fenêtres
test_subset = test_windows[:100]
results = []

for i, test_window in tqdm(enumerate(test_subset), total=len(test_subset)):
    query_vector = test_window["sensor_data"]

    try:
        # Recherche vectorielle
        results_vector = train_table.search(
            vectors={"flat_index": [query_vector]},
            n=5  # Trouver les 5 motifs les plus similaires
        )

        # Vérifier les résultats
        if results_vector and isinstance(results_vector[0], list):
            similar_pannes = [result.get("panne", None) for result in results_vector[0]]
            similar_pannes = [p for p in similar_pannes if p is not None]

            if similar_pannes:
                predicted_panne = max(set(similar_pannes), key=similar_pannes.count)
            else:
                predicted_panne = -1  # Valeur par défaut
        else:
            predicted_panne = -1  # Valeur par défaut

        # Stocker les résultats
        results.append({
            "fenêtre": i,
            "prédiction": predicted_panne,
            "réelle": test_window["panne"]
        })

    except Exception as e:
        print(f"Erreur lors de la prédiction pour la fenêtre {i}: {e}")
        results.append({
            "fenêtre": i,
            "prédiction": -1,  # Valeur par défaut en cas d'erreur
            "réelle": test_window["panne"]
        })

# Créer un DataFrame avec les résultats
results_df = pd.DataFrame(results)

# Calculer la précision globale
accuracy = (results_df["prédiction"] == results_df["réelle"]).mean()

# Compter les fenêtres par classe prédite
class_counts = results_df["prédiction"].value_counts()

# Afficher les résultats
print(f"Précision globale pour les 100 premières fenêtres : {accuracy:.2%}")
print(f"Nombre total de fenêtres testées : {len(results_df)}")
print("\nPrédictions par classe :")
print(class_counts)

# Afficher le DataFrame des résultats pour l'analyse
import ace_tools as tools
tools.display_dataframe_to_user(name="Résultats des 100 Premières Fenêtres", dataframe=results_df)


In [ ]:
################################################################
#### Prédiction pour les 100 premières fenêtres             ####
################################################################

# Limiter les tests aux 100 premières fenêtres
test_subset = test_windows[:10]
results = []

for i, test_window in tqdm(enumerate(test_subset), total=len(test_subset)):
    query_vector = test_window["sensor_data"]

    try:
        # Recherche vectorielle
        results_vector = train_table.search(
            vectors={"flat_index": [query_vector]},
            n=5  # Trouver les 5 motifs les plus similaires
        )

        # Vérifier si des résultats sont retournés
        if results_vector and isinstance(results_vector[0], list):
            similar_pannes = [result.get("panne", None) for result in results_vector[0]]
            similar_pannes = [p for p in similar_pannes if p is not None]

            if similar_pannes:
                predicted_panne = max(set(similar_pannes), key=similar_pannes.count)
            else:
                predicted_panne = -1  # Valeur par défaut si aucune panne valide
        else:
            predicted_panne = -1  # Valeur par défaut si aucun résultat

        # Stocker les résultats
        results.append({
            "fenêtre": i,
            "prédiction": predicted_panne,
            "réelle": test_window["panne"]
        })

    except Exception as e:
        print(f"Erreur lors de la prédiction pour la fenêtre {i}: {e}")
        results.append({
            "fenêtre": i,
            "prédiction": -1,  # Valeur par défaut en cas d'erreur
            "réelle": test_window["panne"]
        })

# Créer un DataFrame avec les résultats
results_df = pd.DataFrame(results)

# Calculer la précision globale
accuracy = (results_df["prédiction"] == results_df["réelle"]).mean()

# Compter les fenêtres par classe prédite
class_counts = results_df["prédiction"].value_counts()

# Afficher les résultats
print(f"Précision globale pour les 100 premières fenêtres : {accuracy:.2%}")
print(f"Nombre total de fenêtres testées : {len(results_df)}")
print("\nPrédictions par classe :")
print(class_counts)

# Afficher le DataFrame avec pandas
print("\nAperçu des résultats :")
print(results_df.head(10))  # Affiche les 10 premières lignes pour validation


In [ ]:
################################################################
#### Prédiction des Pannes pour l'Ensemble de Test Basée    ####
#### Sur le Vote Majoritaire dans l'Ensemble d'Entrainement ####
#### Avec la Classe Définie par la Dernière Valeur dans la  ####
#### Fenêtre                                                ####
################################################################

# Initialiser une liste pour stocker les résultats
results = []

# Parcourir toutes les fenêtres de test
for i, test_window in tqdm(enumerate(test_windows), total=len(test_windows)):
    # Récupérer le vecteur de test
    query_vector = test_window["sensor_data"]

    try:
        # Recherche vectorielle
        results_vector = train_table.search(
            vectors={"flat_index": [query_vector]},
            n=5  # Trouver les 5 motifs les plus similaires
        )

        # Vérifier si des résultats sont retournés
        if results_vector and isinstance(results_vector[0], list):
            # Extraire les valeurs de panne des motifs similaires
            similar_pannes = [result.get("panne", None) for result in results_vector[0]]

            # Filtrer les valeurs valides (évitant les None ou valeurs non numériques)
            similar_pannes = [p for p in similar_pannes if p is not None]

            if similar_pannes:
                # Prédiction par vote majoritaire
                predicted_panne = max(set(similar_pannes), key=similar_pannes.count)
            else:
                # Valeur par défaut si aucune panne similaire valide n'est trouvée
                predicted_panne = -1
        else:
            # Valeur par défaut si aucun résultat n'est retourné
            predicted_panne = -1

        # Ajouter les résultats à la liste
        results.append({
            "fenêtre": i,
            "prédiction": predicted_panne,
            "réelle": test_window['panne']  # Correspond à la dernière valeur dans la fenêtre de test
        })

    except Exception as e:
        print(f"Erreur lors de la prédiction pour la fenêtre {i}: {e}")

# Créer un DataFrame avec les résultats
results_df = pd.DataFrame(results)

# Calculer la précision globale
accuracy = (results_df["prédiction"] == results_df["réelle"]).mean()

# Compter les fenêtres par classe prédite
class_counts = results_df["prédiction"].value_counts()

# Afficher les résultats
print(f"Précision globale sur l'ensemble de test : {accuracy:.2%}")
print(f"Nombre total de fenêtres testées : {len(results_df)}")
print("\nPrédictions par classe :")
print(class_counts)

# Afficher le DataFrame des résultats pour l'analyse
import ace_tools as tools
tools.display_dataframe_to_user(name="Résultats de Prédictions des Fenêtres", dataframe=results_df)


In [10]:
# Parcourir toutes les fenêtres de test
for i, test_window in enumerate(test_windows):
    # Récupérer le vecteur de test
    query_vector = test_window["sensor_data"]

    # Recherche vectorielle
    results = train_table.search(
        vectors={"flat_index": [query_vector]},
        n=5  # Trouver les 5 motifs les plus similaires
    )

    # Extraire les valeurs de panne des motifs similaires
    similar_pannes = results[0]["panne"].tolist()

    # Prédiction par majorité
    predicted_panne = max(set(similar_pannes), key=similar_pannes.count)

    # Affichage simple
    print(f"Fenêtre {i + 1} - Prédiction : {predicted_panne} - Valeur de Test : {test_window['panne']}")


Fenêtre 1 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 2 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 3 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 4 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 5 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 6 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 7 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 8 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 9 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 10 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 11 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 12 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 13 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 14 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 15 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 16 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 17 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 18 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 19 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 20 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 21 - Prédiction : 1 -

Fenêtre 168 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 169 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 170 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 171 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 172 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 173 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 174 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 175 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 176 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 177 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 178 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 179 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 180 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 181 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 182 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 183 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 184 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 185 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 186 - Prédiction : 1 - Valeur de Test : 0
Fenêtre 187 - Prédiction : 1 - Valeur de Test : 0


KeyboardInterrupt: 

In [ ]:
# Parcourir les fenêtres de test à partir de la fenêtre 1130
start_index = 1129  # Correspond à la Fenêtre 1130
for i, test_window in enumerate(test_windows[start_index:], start=start_index + 1):
    # Récupérer le vecteur de test
    query_vector = test_window["sensor_data"]

    # Recherche vectorielle
    results = train_table.search(
        vectors={"flat_index": [query_vector]},
        n=5  # Trouver les 5 motifs les plus similaires
    )

    # Extraire les valeurs de panne des motifs similaires
    similar_pannes = results[0]["panne"].tolist()

    # Prédiction par majorité
    predicted_panne = max(set(similar_pannes), key=similar_pannes.count)

    # Affichage simple
    print(f"Fenêtre {i} - Prédiction : {predicted_panne} - Valeur de Test : {test_window['panne']}")


In [ ]:
# Compter les fenêtres dans l'ensemble d'entraînement
num_train_windows = len(train_windows)

# Compter les fenêtres dans l'ensemble de test
num_test_windows = len(test_windows)

# Affichage
print(f"Nombre de fenêtres dans l'ensemble d'entraînement : {num_train_windows}")
print(f"Nombre de fenêtres dans l'ensemble de test : {num_test_windows}")


In [ ]:
# Indice de la fenêtre 622 (les indices Python commencent à 0)
window_index = 623  # Fenêtre 622 correspond à l'index 621

# Récupérer les informations de la fenêtre
window_622 = test_windows[window_index]

# Affichage des détails
print(f"Fenêtre 622 : {window_622}")


In [ ]:
# Filtrer le DataFrame pour le timestamp spécifique
timestamp_to_find = '2020-07-15 13:43:50'
filtered_row = df[df['timestamp'] == pd.Timestamp(timestamp_to_find)]

# Affichage du résultat
if not filtered_row.empty:
    print("Ligne correspondant au timestamp :")
    display(filtered_row)
else:
    print(f"Aucune ligne trouvée pour le timestamp {timestamp_to_find}")


In [ ]:
# Filtrer le DataFrame pour le timestamp spécifique

filtered_row = df[df['panne'] == 2]

# Affichage du résultat
if not filtered_row.empty:
    print("Ligne correspondant au timestamp :")
    display(filtered_row)
else:
    print(f"Aucune ligne trouvée pour le timestamp {timestamp_to_find}")


In [ ]:
pip install kdbai-client

In [ ]:
import pandas as pd
import numpy as np
# Charger les données
df = pd.read_csv("../Datasources/MetroPT3_imputed_final.csv", delimiter=",", decimal=".", index_col=0)
df.reset_index(drop=True, inplace=True)

<h3 style="text-align: center; font-family: Arial, sans-serif; color: RED;">################# DÉCLARATION / INITIALISATION #################</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
</ul>

In [ ]:
# Convertir timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
#display(df.dtypes)

continuous_features = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]
categorical_features = ["COMP", "DV_eletric", "Towers", "LPS", "Pressure_switch", "Oil_level", "Caudal_impulses"]

# Conserver uniquement les colonnes continues, catégorielles et 'timestamp'
columns_to_keep = ["timestamp", "panne"] + continuous_features + categorical_features
df = df[columns_to_keep]

display(df.head(2))

<h3 style="text-align: center; font-family: Arial, sans-serif; color: RED;">#################### FIN DÉCLARATION #################</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
</ul>

In [ ]:
import kdbai_client as kdbai
session = kdbai.Session(endpoint="https://cloud.kdb.ai/instance/atr0v2owym", api_key="a5b897a7b4-Av/lT//S3W++g8K11vVKfJo5rIj5gK492sbpts/1f+cr/5m5n5JluPcqcQR2gGiiQQ7SrCFAc7ma2RhT")

# create reference to default database
db = session.database('default')

In [ ]:
print(db.tables)
# Return: an empty array if no tables have been created []

In [ ]:
##################################
###### SUPPRIMER LES TABLES ######
##################################
# Accéder aux tables existantes
train_table = db.table("train_sensors")
test_table = db.table("test_sensors")

# Supprimer mes 2 tables
try:
    train_table.drop()
    test_table.drop()
    print("Table 'train_sensors' supprimée avec succès.")
    print("Table 'test_sensors' supprimée avec succès.")
except Exception as e:
    print(f"Erreur lors de la suppression de 'train_sensors' : {e}")

In [ ]:
###################################################################
################ Train : Panne2 & Panne3 & Panne4   ################
################ Test  : Panne1           #########################
###################################################################


train_periods = [{'start': '2020-04-18 23:59:10', 'end': '2020-09-01 03:59:50'}]
test_periods  = [{'start': '2020-02-01 00:00:00', 'end': '2020-04-18 23:59:00'}]

train_df = pd.concat([
    df[(df['timestamp'] >= pd.Timestamp(period['start'])) & 
       (df['timestamp'] <= pd.Timestamp(period['end']))]
    for period in train_periods
])

# Filtrer les périodes pour le test
test_df = pd.concat([
    df[(df['timestamp'] >= pd.Timestamp(period['start'])) & 
       (df['timestamp'] <= pd.Timestamp(period['end']))]
    for period in test_periods
])

print(f"Entraînement : {len(train_df)} lignes")
print(f"Test : {len(test_df)} lignes")


In [ ]:
###################################################################################################
################ Train : Panne1   (1J ET DEMI)                                              #######
################ Test  : Panne4  (15mn + 15mn + 15mn)     #########################################
###################################################################################################


train_periods = [{'start': '2020-04-17 21:30:00', 'end': '2020-04-19 00:15:00'}]
test_periods  = [{'start': '2020-07-15 14:00:00', 'end': '2020-07-15 14:45:00'}]

train_df = pd.concat([
    df[(df['timestamp'] >= pd.Timestamp(period['start'])) & 
       (df['timestamp'] <= pd.Timestamp(period['end']))]
    for period in train_periods
])

# Filtrer les périodes pour le test
test_df = pd.concat([
    df[(df['timestamp'] >= pd.Timestamp(period['start'])) & 
       (df['timestamp'] <= pd.Timestamp(period['end']))]
    for period in test_periods
])

print(f"Entrainement : {len(train_df)} lignes")
print("Données d'entraînement (plage de temps) :")
print(train_df['timestamp'].min(), train_df['timestamp'].max())
print("---------------------------")
print(f"Test : {len(test_df)} lignes")
print("Données de test (plage de temps) :")
print(test_df['timestamp'].min(), test_df['timestamp'].max())


In [ ]:
from tqdm import tqdm

# Paramètres des fenêtres temporelles
window_size = 100  # Taille de la fenêtre
step_size = 1      # Pas de glissement
numeric_columns = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]

# Initialiser les compteurs pour les fenêtres invalides
invalid_train_count = 0
invalid_test_count = 0

# Générer les fenêtres pour l'entraînement avec suivi de la progression
train_windows = []
print("Génération des fenêtres pour l'entraînement...")
for i in tqdm(range(0, len(train_df) - window_size + 1, step_size)):
    # Extraire les données de la fenêtre
    window_data = train_df.iloc[i:i + window_size][numeric_columns].values.flatten()
    if len(window_data) == window_size * len(numeric_columns):  # Validation de la longueur
        train_windows.append({
            "index": i,
            "timestamp": train_df.iloc[i]["timestamp"],
            "sensor_data": window_data.tolist()
        })
    else:
        invalid_train_count += 1  # Compter les fenêtres invalides

# Générer les fenêtres pour le test avec suivi de la progression
test_windows = []
print("\nGénération des fenêtres pour le test...")
for i in tqdm(range(0, len(test_df) - window_size + 1, step_size)):
    # Extraire les données de la fenêtre
    window_data = test_df.iloc[i:i + window_size][numeric_columns].values.flatten()
    if len(window_data) == window_size * len(numeric_columns):  # Validation de la longueur
        test_windows.append({
            "index": i,
            "timestamp": test_df.iloc[i]["timestamp"],
            "sensor_data": window_data.tolist()
        })
    else:
        invalid_test_count += 1  # Compter les fenêtres invalides

# Afficher le nombre de fenêtres générées et invalides
print(f"\nFenêtres d'entraînement : {len(train_windows)}")
print(f"Fenêtres invalides d'entraînement : {invalid_train_count}")
print(f"Fenêtres de test : {len(test_windows)}")
print(f"Fenêtres invalides de test : {invalid_test_count}")


In [ ]:
# Sauvegarder les fenetres localement sur le C

import json

# Convertir les fenêtres pour rendre les Timestamps sérialisables
def convert_windows(windows):
    for window in windows:
        window['timestamp'] = window['timestamp'].isoformat()  # Convertir en chaîne
    return windows

# Convertir et sauvegarder les fenêtres d'entraînement
train_windows_serializable = convert_windows(train_windows)

# Sauvegarder les fenêtres d'entraînement
with open("train_windows.json", "w") as train_file:
    json.dump(train_windows, train_file)
print("TSS_Fenêtres d'entraînement sauvegardées dans train_windows.json.")

# Convertir et sauvegarder les fenêtres de test
test_windows_serializable = convert_windows(test_windows)
# Sauvegarder les fenêtres de test
with open("test_windows.json", "w") as test_file:
    json.dump(test_windows, test_file)
print("TSS_Fenetres de test sauvegardées dans test_windows.json.")


In [ ]:
##################################
###### SUPPRIMER LES TABLES ######
##################################
# Accéder aux tables existantes
train_table = db.table("train_sensors")
test_table = db.table("test_sensors")

# Supprimer mes 2 tables
try:
    train_table.drop()
    test_table.drop()
    print("Table 'train_sensors' supprimée avec succès.")
    print("Table 'test_sensors' supprimée avec succès.")
except Exception as e:
    print(f"Erreur lors de la suppression de 'train_sensors' : {e}")

In [ ]:
# Schéma pour les tables
sensor_schema = [
    {"name": "index", "type": "int64"},               # Identifiant unique
    {"name": "timestamp", "type": "datetime64[ns]"}, # Timestamp
    {"name": "sensor_data", "type": "float64s"}      # Données vectorielles
                ]

# Index pour la recherche vectorielle
indexes = [
    {
        "name": "flat_index",
        "type": "flat",
        "column": "sensor_data",
        "params": {"metric": "L2","dims": 500}
    }
         ]

# Créer les tables
train_table = db.create_table("train_sensors", schema=sensor_schema, indexes=indexes)
test_table = db.create_table("test_sensors", schema=sensor_schema, indexes=indexes)
print("Table 'train_sensors' créée avec succès.")
print("Table 'test_sensors' créée avec succès.")

In [ ]:
from tqdm import tqdm
from kdbai_client import KDBAIException

# Fonction d'insertion des données
def insert_data(table, data_windows, batch_size=100):
    """
    Insère les données dans une table KDB.AI par lots avec une barre de progression.
    """
    total_batches = -(-len(data_windows) // batch_size)  # Calculer le nombre total de batches
    print(f"Insertion dans la table {table.name}... Total de batches : {total_batches}")
    
    with tqdm(total=total_batches, desc=f"Insertion dans {table.name}") as pbar:
        for batch_index, i in enumerate(range(0, len(data_windows), batch_size), start=1):
            batch = data_windows[i:i + batch_size]
            try:
                table.insert(batch)
                pbar.update(1)
            except KDBAIException as e:
                pbar.set_postfix({"Erreur": "Oui"})
                print(f"Erreur lors de l'insertion du batch {batch_index}/{total_batches}: {e}")
                for record in batch:
                    try:
                        table.insert([record])  # Tentative d'insertion individuelle pour débogage
                    except KDBAIException as single_error:
                        print(f"Erreur sur l'enregistrement {record['index']}: {single_error}")

# Insertion des données d'entraînement
insert_data(train_table, train_windows)

# Insertion des données de test
insert_data(test_table, test_windows)

print("Insertion des données terminée.")


In [ ]:
# Compte le nombre de lignes en utilisant la colonne 'index'
print(f"Entrainement : {len(train_df)} lignes")

aggs = {
    "row_count": ["count", "index"],
    "min_timestamp": ["min", "timestamp"],
    "max_timestamp": ["max", "timestamp"],
}

query_result = train_table.query(aggs=aggs)
print(f"Nombre de lignes : {query_result['row_count']}")
print(f"Timestamp minimum : {query_result['min_timestamp']}")
print(f"Timestamp maximum : {query_result['max_timestamp']}")
print("----------------------------")
print(f"Test : {len(test_df)} lignes")
query_result = test_table.query(aggs=aggs)
print(f"Nombre de lignes : {query_result['row_count']}")
print(f"Timestamp minimum : {query_result['min_timestamp']}")
print(f"Timestamp maximum : {query_result['max_timestamp']}")


In [ ]:
# Exemple de prédiction avec une fenêtre de test
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="numpy")


# Sélectionner un vecteur de requête (par exemple, la première fenêtre de test)
query_vector = test_windows[0]["sensor_data"]

# Effectuer une recherche vectorielle dans la table d'entraînement
results = train_table.search(
    vectors={"flat_index": [query_vector]},
    n=5,  # Trouver les 5 motifs les plus similaires
)

# Afficher les résultats de la recherche
print("\nRésultats de recherche vectorielle :")
for i, row in results[0].iterrows():
    print(f"Index d'entraînement : {row['index']}, Distance : {row['__nn_distance']}")
    print(f"Timestamp correspondant : {row['timestamp']}")
    print(f"Motif similaire (extrait des données capteurs) : {row['sensor_data'][:5]} ...")

# Extraire les indices similaires pour les visualiser
similar_indices = results[0]['index']

# Visualiser le motif de test et les motifs similaires
plt.figure(figsize=(10, 6))

# Tracer le vecteur de test
plt.plot(query_vector, marker="o", linestyle="-", label="Motif de Test", color="blue")

# Parcourir les indices similaires
for idx in similar_indices:
    # Requête pour récupérer les vecteurs similaires
    result_row = results[0][results[0]['index'] == idx]
    similar_vector = result_row['sensor_data'].iloc[0]
    
    # Tracer les vecteurs similaires
    plt.plot(similar_vector, marker="o", linestyle="--", alpha=0.7, label=f"Similaire {idx}")

# Ajouter des étiquettes et une légende
plt.xlabel("Timestamp")
plt.ylabel("Valeurs des Capteurs")
plt.title("Comparaison des Motifs : Test vs Similaires")
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
total_batches = len(train_windows) // batch_size + (len(train_windows) % batch_size > 0)
print(f"Nombre de batches attendu : {total_batches}")


In [ ]:
duplicate_rows = train_df[train_df.duplicated()]
print(f"Lignes dupliquées dans train_df : {len(duplicate_rows)}")


In [ ]:
query_result = train_table.query(select=["timestamp"])
db_min_timestamp = query_result["timestamp"].min()
db_max_timestamp = query_result["timestamp"].max()
print(f"Plage temporelle dans la base : {db_min_timestamp} à {db_max_timestamp}")
print(f"Plage temporelle locale : {train_df['timestamp'].min()} à {train_df['timestamp'].max()}")


<h3 style="text-align: center; font-family: Arial, sans-serif; color: BLUE;">################# QUELQUES VALIDATIONS #################</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
</ul>

In [ ]:
print(f"Premiere fenetre train:\n{train_windows[0]}")

In [ ]:
aggs = {
    "row_count": ["count", "index"],
    "min_timestamp": ["min", "timestamp"],
    "max_timestamp": ["max", "timestamp"],
}

query_result = train_table.query(aggs=aggs)
print(f"Nombre de lignes : {query_result['row_count']}")
print(f"Timestamp minimum : {query_result['min_timestamp']}")
print(f"Timestamp maximum : {query_result['max_timestamp']}")

In [ ]:
# Comparer les timestamps entre train_df et KDB.ai
db_timestamps = pd.date_range(start="2020-04-17 21:30:00", end="2020-04-18 23:58:30", freq="10s")
missing_timestamps = train_df[~train_df["timestamp"].isin(db_timestamps)]

print(f"Lignes manquantes dans la base : {len(missing_timestamps)}")
print(missing_timestamps)


In [ ]:
# Préparer les lignes manquantes pour l'insertion
missing_rows = []
for row in missing_timestamps.itertuples(index=True, name="Row"):
    missing_rows.append({
        "index": row.Index,  # Utiliser l'index fourni par itertuples
        "timestamp": row.timestamp,
        "sensor_data": list(row[2:])  # Inclure les colonnes numériques sous forme de liste
    })

# Insérer les données manquantes dans la table train_sensors
batch_size = 10  # Taille du batch pour l'insertion
print(f"Réinsertion des {len(missing_rows)} lignes manquantes...")

for i in range(0, len(missing_rows), batch_size):
    batch = missing_rows[i:i + batch_size]
    try:
        train_table.insert(batch)
        print(f"Batch {i // batch_size + 1}/{-(-len(missing_rows) // batch_size)} inséré avec succès.")
    except Exception as e:
        print(f"Erreur lors de l'insertion du batch {i // batch_size + 1} : {e}")

print("Réinsertion des données manquantes terminée.")


In [ ]:
# Vérifiez les colonnes utilisées pour la création de sensor_data
numeric_columns = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]  # Exemple
print("Colonnes utilisées pour sensor_data :", numeric_columns)

# Préparer les lignes manquantes pour l'insertion
missing_rows = []
for row in missing_timestamps.itertuples(index=True, name="Row"):
    # Vérifiez les données avant de les ajouter
    try:
        sensor_data = [getattr(row, col) for col in numeric_columns]  # Extraire les colonnes nécessaires
        missing_rows.append({
            "index": row.Index,  # Utiliser l'index fourni par itertuples
            "timestamp": row.timestamp,
            "sensor_data": sensor_data  # Liste de données numériques
        })
    except AttributeError as e:
        print(f"Erreur lors de la préparation des données pour l'index {row.Index} : {e}")

# Réinsertion dans la base
batch_size = 10
print(f"Réinsertion des {len(missing_rows)} lignes manquantes...")

for i in range(0, len(missing_rows), batch_size):
    batch = missing_rows[i:i + batch_size]
    try:
        train_table.insert(batch)
        print(f"Batch {i // batch_size + 1}/{-(-len(missing_rows) // batch_size)} inséré avec succès.")
    except Exception as e:
        print(f"Erreur lors de l'insertion du batch {i // batch_size + 1} : {e}")

print("Réinsertion des données manquantes terminée.")


In [ ]:
# Vérification des dimensions de chaque vecteur avant l'insertion
expected_length = 500  # Longueur attendue des vecteurs

invalid_vectors = []
for row in missing_rows:
    if len(row["sensor_data"]) != expected_length:
        invalid_vectors.append(row)

# Si des vecteurs invalides sont trouvés, les afficher
if invalid_vectors:
    print(f"Vecteurs invalides détectés : {len(invalid_vectors)}")
    for idx, invalid_row in enumerate(invalid_vectors[:5]):  # Afficher les 5 premiers vecteurs invalides
        print(f"Index : {invalid_row['index']}, Longueur : {len(invalid_row['sensor_data'])}")
else:
    print("Tous les vecteurs ont la bonne dimension.")

# Réinsertion uniquement des vecteurs valides
valid_rows = [row for row in missing_rows if len(row["sensor_data"]) == expected_length]

print(f"Réinsertion des {len(valid_rows)} lignes valides...")
for i in range(0, len(valid_rows), batch_size):
    batch = valid_rows[i:i + batch_size]
    try:
        train_table.insert(batch)
        print(f"Batch {i // batch_size + 1}/{-(-len(valid_rows) // batch_size)} inséré avec succès.")
    except Exception as e:
        print(f"Erreur lors de l'insertion du batch {i // batch_size + 1} : {e}")

print("Réinsertion des données valides terminée.")


In [ ]:
# Taille réelle de la première fenêtre
print(f"Taille des données dans la première fenêtre d'entraînement : {len(train_windows[0]['sensor_data'])}")
print(f"Taille des données dans la première fenêtre de test : {len(test_windows[0]['sensor_data'])}")

# Cela correspond à window_size (100) × nombre de colonnes (numeric_columns, soit 5).

print('--------------------------------')
# Vérifier la taille des fenêtres générées
for i in range(5):  # Tester sur les 5 premières fenêtres
    actual_window = train_df.iloc[i:i + window_size][numeric_columns]
    print(f"Fenêtre {i} - Taille réelle : {len(actual_window)}")
    

In [ ]:
# Initialiser des compteurs pour détecter les erreurs
invalid_index_count = 0
invalid_timestamp_count = 0
invalid_sensor_data_count = 0

# Parcourir toutes les fenêtres pour vérifier les types et formats
for i, window in enumerate(train_windows):
    # Vérifier l'index
    if not isinstance(window['index'], int):
        print(f"Fenêtre {i} : 'index' non valide ({type(window['index'])})")
        invalid_index_count += 1
    
    # Vérifier le timestamp
    try:
        pd.Timestamp(window['timestamp'])  # Vérifie si le timestamp est convertible
    except Exception as e:
        print(f"Fenêtre {i} : 'timestamp' non valide ({window['timestamp']})")
        invalid_timestamp_count += 1
    
    # Vérifier sensor_data
    if not isinstance(window['sensor_data'], list) or len(window['sensor_data']) != 500:
        print(f"Fenêtre {i} : 'sensor_data' non valide (Longueur : {len(window['sensor_data'])})")
        invalid_sensor_data_count += 1
    elif not all(isinstance(value, float) for value in window['sensor_data']):
        print(f"Fenêtre {i} : 'sensor_data' contient des valeurs non flottantes.")
        invalid_sensor_data_count += 1

# Résumé des erreurs détectées
print(f"Erreurs dans 'index' : {invalid_index_count}")
print(f"Erreurs dans 'timestamp' : {invalid_timestamp_count}")
print(f"Erreurs dans 'sensor_data' : {invalid_sensor_data_count}")

In [ ]:
# Tester l'insertion avec un petit lot

# Schéma pour les tables
sensor_schema = [
    {"name": "index", "type": "int64"},               # Identifiant unique
    {"name": "timestamp", "type": "datetime64[ns]"}, # Timestamp
    {"name": "sensor_data", "type": "float64s"}      # Données vectorielles
]

# Index pour la recherche vectorielle
indexes = [
    {
        "name": "flat_index",
        "type": "flat",
        "column": "sensor_data",
        "params": {"metric": "L2","dims": 500}
    }
]

# Créer les tables
train_table = db.create_table("train_sensors", schema=sensor_schema, indexes=indexes)
test_table = db.create_table("test_sensors", schema=sensor_schema, indexes=indexes)


single_window = [train_windows[0]]  # Utiliser uniquement la première fenêtre
try:
    train_table.insert(single_window)
    print("Insertion réussie pour une seule fenêtre.")
except Exception as e:
    print(f"Erreur lors de l'insertion d'une seule fenêtre : {e}")

In [ ]:
from datetime import datetime

try:
    datetime.fromisoformat(single_window[0]['timestamp'])
    print("Le format du timestamp est correct.")
except ValueError as e:
    print(f"Problème avec le format du timestamp : {e}")


In [ ]:
for value in single_window[0]['sensor_data']:
    if not isinstance(value, float):
        print(f"Problème détecté : {value} de type {type(value)}")


In [ ]:
if len(single_window[0]['sensor_data']) != 500:
    print("La longueur de sensor_data n'est pas correcte.")


In [ ]:
print("Schéma de la table :")
print(train_table.schema)


In [ ]:
from datetime import datetime

for window in train_windows:
    window['timestamp'] = datetime.fromisoformat(window['timestamp'])


In [ ]:
train_table.query(limit=5)  # Affiche les 5 premières lignes


In [ ]:
test_row_count = test_table.query(aggs=aggs)
print(f"Lignes stockées dans la table de test : {test_row_count}")


In [ ]:
# Afficher le contenu du lot envoyé pour inspection
print("Contenu du lot de test :")
for i, window in enumerate(test_batch):
    print(f"Fenêtre {i}: {window}")


In [ ]:
# Vérification approfondie des valeurs
for i, window in enumerate(test_batch):
    if not all(isinstance(v, float) and not (np.isnan(v) or np.isinf(v)) for v in window['sensor_data']):
        print(f"Fenêtre {i} contient des valeurs invalides.")


In [ ]:
# Remplacer les valeurs invalides par une valeur par défaut (ex. : 0.0)
for window in train_windows:
    window['sensor_data'] = [
        0.0 if (np.isnan(v) or np.isinf(v)) else v
        for v in window['sensor_data']
    ]


In [ ]:
# Vérifier la structure des 5 premières fenêtres
for window in train_windows[:5]:
    print(f"Index : {window['index']}")
    print(f"Timestamp : {window['timestamp']}")
    print(f"Longueur de sensor_data : {len(window['sensor_data'])}")
    print(f"Exemple de données : {window['sensor_data'][:5]}")


In [ ]:
# Vérification des longueurs des fenêtres
for i, window in enumerate(train_windows):
    if len(window['sensor_data']) != window_size * len(numeric_columns):
        print(f"Fenêtre {i} a une taille incorrecte : {len(window['sensor_data'])}")
print("Validation de la longueur des fenêtres terminée.")

In [ ]:
# Vérification de la continuité temporelle
for i, window in enumerate(train_windows):
    timestamps = train_df.iloc[window['index']:window['index'] + window_size]['timestamp']
    time_diffs = timestamps.diff().dt.total_seconds().dropna()
    if not all(time_diffs == 10):  # Vérifie si tous les gaps sont de 10 secondes
        print(f"Fenêtre {i} a des timestamps non consécutifs.")
print("Validation de la continuité temporelle terminée.")

In [ ]:
# Exemple : Vérifier la réponse brute
response = train_table.insert(train_windows[:5])
print("Réponse brute :", response)

In [ ]:
print(db.tables)

In [ ]:
# Requête pour compter les lignes dans une table
train_row_count = db.query("SELECT COUNT(*) AS row_count FROM train_sensors").iloc[0]['row_count']
print(f"Lignes stockées dans la table d'entraînement : {train_row_count}")

test_row_count = db.query("SELECT COUNT(*) AS row_count FROM test_sensors").iloc[0]['row_count']
print(f"Lignes stockées dans la table de test : {test_row_count}")


In [ ]:
# Afficher toutes les méthodes et attributs de l'objet db
print(dir(db.tables))

In [ ]:
# Afficher toutes les méthodes et attributs de l'objet train_table
print(dir(train_table))

In [ ]:
# Obtenir les détails sur la méthode query
help(train_table.query)

In [ ]:
# Définir l'agrégation pour compter les lignes
aggs = {'row_count': ['count', 'index']}  # Compte le nombre de lignes en utilisant la colonne 'index'

# Effectuer la requête pour compter les lignes
train_row_count = train_table.query(aggs=aggs)
print(f"Lignes stockées dans la table d'entraînement : {train_row_count}")


In [ ]:
# Parametres des fenetres temporelles
window_size = 100  # Taille de la fenetre
step_size = 1      # Pas de glissement
numeric_columns = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]

# Générer les fenetres pour l'entrainement
train_windows = [
    {
        "index": i,
        "timestamp": train_df.iloc[i]["timestamp"],
        "sensor_data": train_df.iloc[i:i + window_size][numeric_columns].values.flatten().tolist()
    }
    for i in range(0, len(train_df) - window_size + 1, step_size)
]

# Générer les fenetres pour le test
test_windows = [
    {
        "index": i,
        "timestamp": test_df.iloc[i]["timestamp"],
        "sensor_data": test_df.iloc[i:i + window_size][numeric_columns].values.flatten().tolist()
    }
    for i in range(0, len(test_df) - window_size + 1, step_size)
]

print(f"Fenêtres d'entraînement : {len(train_windows)}")
print(f"Fenêtres de test : {len(test_windows)}")


In [ ]:
from sklearn.decomposition import PCA

numeric_columns = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]

# Construire les vecteurs pour chaque fenetre (numeric_columns)
window_vectors = np.array([
    window[numeric_columns].values.flatten() for window in windows
])

# Appliquer PCA pour réduire la dimensionnalité
pca = PCA(n_components=8)  # Réduire a 8 dimensions
compressed_vectors = pca.fit_transform(window_vectors)

print(f"Dimensions des vecteurs compressés : {compressed_vectors.shape}")
print(f"Vecteur compressé pour la première fenêtre : {compressed_vectors[0]}")